In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dir_path = '/content/drive/MyDrive/DataScience/SIGNATE/債務不履行リスクの低減/データ/'
df_train = pd.read_csv(dir_path + 'train.csv')
df_test = pd.read_csv(dir_path + 'test.csv')

In [3]:
df_train.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
0,0,609.296068,3 years,8.421982,A5,0 years,debt_consolidation,714.061803,Individual,FullyPaid
1,1,1183.266999,5 years,10.286776,B1,10 years,credit_card,697.706701,Individual,ChargedOff
2,2,695.783256,3 years,14.723425,C2,1 year,debt_consolidation,656.419357,Individual,FullyPaid
3,3,738.392546,3 years,14.260708,C1,0 years,credit_card,657.906852,Individual,FullyPaid
4,4,1642.400654,5 years,25.217452,E5,10 years,debt_consolidation,662.972297,Individual,FullyPaid


In [4]:
df_test.head()

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type
0,242150,1161.830205,3 years,13.105362,C1,10 years,credit_card,682.016083,Individual
1,242151,2257.465815,5 years,14.196200,C5,10 years,debt_consolidation,668.240313,Individual
2,242152,2182.137002,3 years,8.307772,A5,10 years,credit_card,703.277694,Individual
3,242153,520.284912,3 years,6.956504,A2,7 years,debt_consolidation,731.138779,Individual
4,242154,2303.349549,3 years,11.658497,B3,6 years,debt_consolidation,698.368991,Individual


In [5]:
df_train.isnull().sum()

id                   0
loan_amnt            0
term                 0
interest_rate        0
grade                0
employment_length    0
purpose              0
credit_score         0
application_type     0
loan_status          0
dtype: int64

In [6]:
df_train_ = df_train.copy()
df_train_['term'] = df_train_['term'].str.replace(' years','')
df_train_['term'] = df_train_['term'].astype(int)
df_train_['employment_length'] = df_train_['employment_length'].str.replace(' years','')
df_train_['employment_length'] = df_train_['employment_length'].str.replace(' year','')
df_train_['employment_length'] = df_train_['employment_length'].astype(int)
df_train_['application_type'] = df_train_['application_type'].replace({'Individual':0,'Joint App':1})
df_train_['loan_status'] = df_train_['loan_status'].replace({'FullyPaid':0,'ChargedOff':1})

df_train_['grade_rank'] = df_train_['grade'].str.extract('([A-Z]+)').astype(str)
df_train_['grade_num'] = df_train_['grade'].str.extract('([0-9]+)').astype(int)
df_train_['grade_rank_num'] = df_train_['grade_rank'].replace({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5})
df_train_['grade_rank_num'] = df_train_['grade_rank_num'].astype(int)
df_train_['grade_val'] = (df_train_['grade_rank_num']*5) + df_train_['grade_num']

df_train_ = df_train_.drop({'id','grade','grade_rank','grade_num','grade_rank_num'},axis=1)
df_train_

,loan_amnt,term,interest_rate,employment_length,purpose,credit_score,application_type,loan_status,grade_val
0,609.296068,3,8.421982,0,debt_consolidation,714.061803,0,0,5
1,1183.266999,5,10.286776,10,credit_card,697.706701,0,1,6
2,695.783256,3,14.723425,1,debt_consolidation,656.419357,0,0,12
3,738.392546,3,14.260708,0,credit_card,657.906852,0,0,11
4,1642.400654,5,25.217452,10,debt_consolidation,662.972297,0,0,25
...,...,...,...,...,...,...,...,...,...
242145,2427.939262,3,13.673803,8,debt_consolidation,657.206417,0,0,9
242146,1224.346665,3,11.793595,10,debt_consolidation,699.763028,0,1,8
242147,2089.513458,3,8.218123,1,other,724.556339,0,0,4
242148,3459.367114,3,14.875958,9,debt_consolidation,688.249378,0,0,12


In [7]:
df_test_ = df_test.copy()
df_test_['term'] = df_test_['term'].str.replace(' years','')
df_test_['term'] = df_test_['term'].astype(int)
df_test_['employment_length'] = df_test_['employment_length'].str.replace(' years','')
df_test_['employment_length'] = df_test_['employment_length'].str.replace(' year','')
df_test_['employment_length'] = df_test_['employment_length'].astype(int)
df_test_['application_type'] = df_test_['application_type'].replace({'Individual':0,'Joint App':1})

df_test_['grade_rank'] = df_test_['grade'].str.extract('([A-Z]+)').astype(str)
df_test_['grade_num'] = df_test_['grade'].str.extract('([0-9]+)').astype(int)
df_test_['grade_rank_num'] = df_test_['grade_rank'].replace({'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5})
df_test_['grade_rank_num'] = df_test_['grade_rank_num'].astype(int)
df_test_['grade_val'] = (df_test_['grade_rank_num']*5) + df_test_['grade_num']

df_test_ = df_test_.drop({'id','grade','grade_rank','grade_num','grade_rank_num'},axis=1)
df_test_.head()

,loan_amnt,term,interest_rate,employment_length,purpose,credit_score,application_type,grade_val
0,1161.830205,3,13.105362,10,credit_card,682.016083,0,11
1,2257.465815,5,14.196200,10,debt_consolidation,668.240313,0,15
2,2182.137002,3,8.307772,10,credit_card,703.277694,0,5
3,520.284912,3,6.956504,7,debt_consolidation,731.138779,0,2
4,2303.349549,3,11.658497,6,debt_consolidation,698.368991,0,8


In [8]:
df = pd.concat([df_train_, df_test_])
df

,loan_amnt,term,interest_rate,employment_length,purpose,credit_score,application_type,loan_status,grade_val
0,609.296068,3,8.421982,0,debt_consolidation,714.061803,0,0.0,5
1,1183.266999,5,10.286776,10,credit_card,697.706701,0,1.0,6
2,695.783256,3,14.723425,1,debt_consolidation,656.419357,0,0.0,12
3,738.392546,3,14.260708,0,credit_card,657.906852,0,0.0,11
4,1642.400654,5,25.217452,10,debt_consolidation,662.972297,0,0.0,25
...,...,...,...,...,...,...,...,...,...
26895,1169.767130,3,13.609916,1,credit_card,656.939828,0,NaN,9
26896,2235.153322,3,7.248690,10,credit_card,765.770393,0,NaN,2
26897,3272.971410,3,9.957002,6,debt_consolidation,793.887391,0,NaN,9
26898,1607.397707,3,10.892554,10,debt_consolidation,658.815541,0,NaN,8


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['purpose'])
df['purpose'] = le.transform(df['purpose'])
df['purpose'].unique()

array([2, 1, 6, 8, 3, 0, 5, 9, 4, 7])

In [10]:
df_train_ = df[0:242149]
df_train_

,loan_amnt,term,interest_rate,employment_length,purpose,credit_score,application_type,loan_status,grade_val
0,609.296068,3,8.421982,0,2,714.061803,0,0.0,5
1,1183.266999,5,10.286776,10,1,697.706701,0,1.0,6
2,695.783256,3,14.723425,1,2,656.419357,0,0.0,12
3,738.392546,3,14.260708,0,1,657.906852,0,0.0,11
4,1642.400654,5,25.217452,10,2,662.972297,0,0.0,25
...,...,...,...,...,...,...,...,...,...
242144,1022.965398,3,6.568938,1,2,791.296767,0,0.0,1
242145,2427.939262,3,13.673803,8,2,657.206417,0,0.0,9
242146,1224.346665,3,11.793595,10,2,699.763028,0,1.0,8
242147,2089.513458,3,8.218123,1,8,724.556339,0,0.0,4


In [11]:
df_test_ = df[242150:]
df_test_ = df_test_.drop('loan_status',axis=1)
df_test_

,loan_amnt,term,interest_rate,employment_length,purpose,credit_score,application_type,grade_val
0,1161.830205,3,13.105362,10,1,682.016083,0,11
1,2257.465815,5,14.196200,10,2,668.240313,0,15
2,2182.137002,3,8.307772,10,1,703.277694,0,5
3,520.284912,3,6.956504,7,2,731.138779,0,2
4,2303.349549,3,11.658497,6,2,698.368991,0,8
...,...,...,...,...,...,...,...,...
26895,1169.767130,3,13.609916,1,1,656.939828,0,9
26896,2235.153322,3,7.248690,10,1,765.770393,0,2
26897,3272.971410,3,9.957002,6,2,793.887391,0,9
26898,1607.397707,3,10.892554,10,2,658.815541,0,8


In [12]:
x = df_train_.drop('loan_status',axis=1)
x.head()

,loan_amnt,term,interest_rate,employment_length,purpose,credit_score,application_type,grade_val
0,609.296068,3,8.421982,0,2,714.061803,0,5
1,1183.266999,5,10.286776,10,1,697.706701,0,6
2,695.783256,3,14.723425,1,2,656.419357,0,12
3,738.392546,3,14.260708,0,1,657.906852,0,11
4,1642.400654,5,25.217452,10,2,662.972297,0,25


In [13]:
y = df_train_['loan_status']
y

0         0.0
1         1.0
2         0.0
3         0.0
4         0.0
         ... 
242144    0.0
242145    0.0
242146    1.0
242147    0.0
242148    0.0
Name: loan_status, Length: 242149, dtype: float64

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [15]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test)

In [16]:
#light gbmモデル構築
params = {
    'objective':'binary',
    'seed':0
    # 'learning_rate':0.61,
    # 'verbose':0,
    # 'max_depth':20,
    # 'metrics':'binary_logloss',
    # 'num_leaves':40,
    # 'early_stopping_round':1000
    # 'bagging_freq':2,
    # 'bagging_fraction':0.3
    # 'min_data_in_leaf':20
    # 'colsample_bytree':0.96
    # 'subsample':0.1
    # 'subsample_freq':100
    # 'min_child_samples':20
}

model = lgb.train(
    params,
    lgb_train,
    valid_sets = lgb_eval,
    num_boost_round = 100,
)

[LightGBM] [Info] Number of positive: 29759, number of negative: 139745
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.105245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 817
[LightGBM] [Info] Number of data points in the train set: 169504, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.175565 -> initscore=-1.546688
[LightGBM] [Info] Start training from score -1.546688


In [17]:
pre = model.predict(x_test)

for i in range(72645):
    if pre[i] >= 0.199:
        pre[i] = 1
    else:
        pre[i] = 0
pre[:50]

array([1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [18]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pre)
f1

0.397504129198018

In [23]:
con = pd.DataFrame(y_test)
con['pre'] = pre
pd.crosstab(con['loan_status'], con['pre'])

pre,0.0,1.0
loan_status,,
0.0,42083,17965
1.0,5016,7581


In [19]:
con = pd.DataFrame(y_test)
con['pre'] = pre
pd.crosstab(con['loan_status'], con['pre'])

pre,0.0,1.0
loan_status,,
0.0,42083,17965
1.0,5016,7581


In [20]:
prediction = model.predict(df_test_)

for i in range(26900):
    if prediction[i] >= 0.199:
        prediction[i] = 1
    else:
        prediction[i] = 0

prediction = prediction.astype(int)
prediction[:50]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0])

In [21]:
submission = df_test_.copy()
submission['id'] = df_test['id']
submission['loan_status'] = prediction
submission = submission.drop({'loan_amnt','term','interest_rate',
                              'employment_length','credit_score',
                              'application_type','purpose','grade_val'},axis=1)
submission.head()

,id,loan_status
0,242150,0
1,242151,1
2,242152,0
3,242153,0
4,242154,0


In [22]:
submission.to_csv('submission_lgbm_5.csv',index=False,header=False)